In [236]:
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup

# specify the url
import urllib.request


In [237]:
#!/usr/bin/env python

try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import certifi
import json

def get_data(url, stock):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)




In [238]:
def get_income_data(url, stock):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)



In [239]:
NIR = [2]
EBITDA = [10]
DTE = [.7]
EPS = [1.2]
GPR = [.7]
PTBV = [1]
PTS = [1]
prices = {}
api_key = "87f2035b04c9137524da80d59c6c7083"

In [244]:
def stockInfo(stock):
    print(stock)
    income_url = "https://financialmodelingprep.com/api/v3/income-statement/" + stock + "?apikey=" + api_key + "&limit=120"
    income_data = get_data(income_url, stock)
    balance_url = "https://financialmodelingprep.com/api/v3/balance-sheet-statement/" + stock + "?apikey=" + api_key + "&limit=120"
    balance_data = get_data(balance_url, stock)
    price_url = "https://financialmodelingprep.com/api/v3/historical-price-full/" + stock + "?apikey=" + api_key + "&limit=120"
    price_data = get_data(price_url, stock)
    old_price = price_data["historical"][-1]["close"]
    new_price = price_data["historical"][0]["close"]
    liabilites = balance_data[4]['totalCurrentLiabilities']
    equity = balance_data[4]['totalStockholdersEquity']
    assets = balance_data[4]['totalAssets']
    shares = balance_data[4]['commonStock']
    revenue = income_data[4]['revenue']
    net_income_ratio = income_data[4]['netIncomeRatio']
    ebitda_ratio = income_data[4]['ebitdaratio']
    eps = income_data[4]['epsdiluted']
    gpr = income_data[4]['grossProfitRatio']
    price_to_book_value = old_price / ((assets - liabilites) / equity)
    price_to_sales = (shares * equity) / revenue
    debt_to_equity = liabilites/equity
    NIR.append(net_income_ratio)
    EBITDA.append(ebitda_ratio)
    DTE.append(debt_to_equity)
    EPS.append(eps)
    GPR.append(gpr)
    PTBV.append(price_to_book_value)
    PTS.append(price_to_sales)
    prices[stock] = (old_price, new_price)



In [245]:
def normalize(arr):
	mn = min(arr)
	mx = max(arr)
	normed = []
	for i in arr:
		standard = (i - mn)/(mx-mn)
		normed.append(standard)
	return normed

In [246]:
def score(stocks, ndte, nnir, nebitda, ngpr, neps, nptbv, npts):
    scores = {}
    for i in range(len(stocks)):
        ticker = stocks[i]
        print(ticker)
        dteScore = abs(ndte[0] - ndte[i + 1])
        nirScore = abs(nnir[0] - nnir[i+ 1])
        ebitdaScore = abs(nebitda[0] - nebitda[i+ 1])
        gprScore = abs(ngpr[0] - ngpr[i+ 1])
        epsScore = abs(neps[0] - neps[i+ 1])
        ptbvScore = abs(nptbv[0] - nptbv[i + 1])
        ptsScore = abs(npts[0] - npts[i+ 1])
        scoreVal = dteScore + nirScore + ebitdaScore + gprScore + epsScore + ptbvScore + ptsScore
        scores[ticker] = (scoreVal)
    return scores

In [247]:
large_cap_stocks = ["UNH", "JNJ", 'XOM', 'JPM', 'PG', 'LLY', 'CVX', 'PFE', 
          'ABBV', 'PEP', 'MRK', 'KO', 'BAC', 'WMT', 'AVGO', 'ABT', 'NEE', 'WFC', 'BMY']
small_cap_stocks = ["TRGP", "ENTG", "WOLF", "FWONK", "TECH", "PTC", "NDSN", "TPL", "DAR", "ELS", 
                   "FICO", "RGEN", "NBIX", "GGG", "PCTY", "SWAV", "SRPT", "AMH", "TTC", "CUBE"]
for i in large_cap_stocks:
    stockInfo(i)

UNH


<ipython-input-237-16be51af4abb>:25: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


JNJ
XOM
JPM
PG
LLY
CVX
PFE
ABBV
PEP
MRK
KO
BAC
WMT
AVGO
ABT
NEE
WFC
BMY


In [248]:
ndte = normalize(DTE)
nnir = normalize(NIR)
nebitda = normalize(EBITDA)
ngpr = normalize(GPR)
neps = normalize(EPS)
nptbv = normalize(PTBV)
npts = normalize(PTS)

In [249]:
scores = score(large_cap_stocks, ndte, nnir, nebitda, ngpr, neps, nptbv, npts)
sortedScores = sorted(scores.items(), key=lambda x: x[1])
print(small_cap_stocks)
print(sortedScores)

UNH
JNJ
XOM
JPM
PG
LLY
CVX
PFE
ABBV
PEP
MRK
KO
BAC
WMT
AVGO
ABT
NEE
WFC
BMY
['TRGP', 'ENTG', 'WOLF', 'FWONK', 'TECH', 'PTC', 'NDSN', 'TPL', 'DAR', 'ELS', 'FICO', 'RGEN', 'NBIX', 'GGG', 'PCTY', 'SWAV', 'SRPT', 'AMH', 'TTC', 'CUBE']
[('MRK', 2.2768507466618515), ('BMY', 2.29959535988188), ('NEE', 2.314594368989606), ('PFE', 2.323246107484023), ('KO', 2.5221764296899343), ('LLY', 2.61007789427595), ('ABT', 2.6533968855514765), ('JNJ', 2.7200252428932665), ('PEP', 2.893506790231655), ('PG', 2.9070994068488076), ('ABBV', 2.9922727937135996), ('WFC', 3.306423667218212), ('WMT', 3.3214155369463625), ('XOM', 3.4869934398631877), ('JPM', 3.5302142734537467), ('AVGO', 3.5823433591423113), ('CVX', 3.6256731246711498), ('BAC', 3.994532617788851), ('UNH', 4.537963827534131)]


In [250]:
numStocks = 5
amt = 5000
investment = amt / numStocks
print(prices)
ret = 0
for i in range(numStocks):
    percent = (prices[sortedScores[i][0]][1] - prices[sortedScores[i][0]][0]) / prices[sortedScores[i][0]][0]
    print(str(sortedScores[i][0]) + " % change: " + str(percent))
    ret += (1 + percent) * investment
print("total return")   
print(ret)
print("total percent change")
print ((ret - amt) / amt)

{'UNH': (212.6000061, 543.17), 'JNJ': (141.7799988, 172.21), 'XOM': (83.7099991, 107.14), 'JPM': (101.7699966, 124.11), 'PG': (87.0400009, 131.78), 'LLY': (83.8600006, 357.86), 'CVX': (113.5400009, 177.09), 'PFE': (33.7760925, 46.06), 'ABBV': (91.9300003, 152.51), 'PEP': (110.5999985, 179.07), 'MRK': (55.5725174, 98.41), 'KO': (46.0699997, 59.39), 'BAC': (27.7999992, 35.7), 'WMT': (88.1699982, 141.14), 'AVGO': (252.8999939, 461.18), 'ABT': (55.3699989, 98.42), 'NEE': (38.9599991, 75.6), 'WFC': (55.8699989, 45.7), 'BMY': (59.9399986, 74.45)}
MRK % change: 0.7708393393746095
BMY % change: 0.24207543775284637
NEE % change: 0.9404517902055085
PFE % change: 0.3636864595867626
KO % change: 0.2891252525881828
total return
7606.17827950791
total percent change
0.5212356559015819


In [251]:
def risk (x):
	url = "https://yh-finance.p.rapidapi.com/stock/v2/get-analysis"

	querystring = {"symbol":x,"region":"US"}

	headers = {
		"X-RapidAPI-Host": "yh-finance.p.rapidapi.com",
		"X-RapidAPI-Key": "ea0fe23a72mshbfc9dfafe36fc9dp1870e7jsnbb7a2a7713ff"
	}

	response = requests.request("GET", url, headers=headers, params=querystring)
	data = json.loads(response.text)
	try:
		beta = data['summaryDetail']['beta']['raw']
		print(beta)
		return beta
	except:
		return -1


In [253]:
keys = []
for i in range(0,5):
	keys.append(sortedScores[i][0])
print(keys)
totalbeta = 0
numBeta = 0
for i in keys:
	beta = risk(i)
	if(beta != -1):
		totalbeta += beta
		numBeta += 1
print(totalbeta/numBeta)



['MRK', 'BMY', 'NEE', 'PFE', 'KO']
0.328297
0.364406
0.493367
0.637067
0.572218
0.47907099999999997
